# Ray Concepts - Task Parallelism (Part 2)

The previous lesson explored Ray's core concepts and how they work. We learned how to define Ray _tasks_, run them, and retrieve the results. We also started learning about how Ray schedules tasks in a distributed environment.

This lesson completes the discussion of Ray tasks by exploring how task dependencies are handled. We'll finish with a look under the hood at Ray's architecture and runtime behavior.

> **Tip:** Recall that the [Ray Package Reference](https://ray.readthedocs.io/en/latest/package-ref.html) in the [Ray Docs](https://ray.readthedocs.io/en/latest/) is useful for exploring the API features we'll learn.

In [1]:
# Imports and initialize Ray. We're adding NumPy for the examples and the tutorial `util` library:

import ray, time, sys    # New notebook, so new process
import numpy as np       # Used for examples
sys.path.append('..')    # Import our own libraries starting in the project root directory

from util.printing import pnd, pd

In [2]:
ray.init(ignore_reinit_error=True)

2020-04-09 11:02:27,058	INFO resource_spec.py:204 -- Starting Ray with 4.69 GiB memory available for workers and up to 2.36 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-04-09 11:02:27,410	INFO services.py:1146 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '192.168.1.149',
 'redis_address': '192.168.1.149:26031',
 'object_store_address': '/tmp/ray/session_2020-04-09_11-02-27_051258_29191/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-04-09_11-02-27_051258_29191/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-04-09_11-02-27_051258_29191'}

Let's work with a new remote function. Previously, our `expensive` and `expensive_task` functions returned tuples that included time durations. Obviously the durations were useful for understanding how long the functions took to execute. Now, it will be more convenient to not return "metadata" like this, but just data values that we care about, because we are going to pass them to other functions. 

Hence, we'll define _dependentcy_ relationships between tasks. We'll learn how Ray handles these dependent, asynchronous computations.

So, let's define a task to return a random NumPy array of some size `n`. As before, we'll add a sleep time, one tenth the size of `n`:

In [3]:
@ray.remote
def make_array(n):
    time.sleep(n/10.0)
    return np.random.standard_normal(n)

Now define a task that can add two NumPy arrays together. The arrays need to be the same size, but we'll ignore any checking for this requirement.

In [4]:
@ray.remote
def add_arrays(a1, a2):
    time.sleep(a1.size/10.0)
    return np.add(a1, a2)

Now lets use them!

In [5]:
start = time.time()
id1 = make_array.remote(20)
id2 = make_array.remote(20)
id3 = add_arrays.remote(id1, id2)
print(ray.get(id3))
pd(time.time() - start, prefix="Total time:")

[ 1.19359074  0.80318181  2.59254066 -0.03606983 -0.64729178 -0.29704776
 -1.58681142 -0.84438872 -1.18256379  1.7343043  -0.86034226  1.5004233
  0.73785033 -0.83608832  0.02295084 -1.21924252 -0.87437391 -2.08081722
 -0.40505855  0.61635172]
Total time: duration:  4.028 seconds


Something subtle and "magical" happened here; when we called `add_arrays`, we didn't need to call `ray.get()` first for `id1` and `id2`, since `add_arrays` expects NumPy arrays. Because `add_arrays` is a Ray task, Ray automatically does the extraction for us, so we can write code that looks more natural.

Furthermore, note that the `add_arrays` task effectively depends on the outputs of the two `make_array` tasks. Ray won't run `add_arrays` until the other tasks are finished. Hence, _Ray handles task dependencies automatically for us._ 

This is why the elapsed time is about 4 seconds. We used a size of 20, so we slept 2 seconds in each call to `make_array`, but those happened in parallel, _followed_ by a second sleep of 2 seconds in `add_arrays`.

Recall from the previous lesson that we explored when to call `ray.get()` to avoid forcing tasks to become synchronous when they should be asynchronous. This additional example illustrates two key points:

* _Don't ask for results you don't need._
* _Don't ask for the results you need until you really need them._

We don't need to see the objects for `id1` and `id2`. We only need the final array for `id3`.

## Using ray.wait() with ray.get()

We've seen several examples of the best idiomatic way to use `ray.get()`. Here again is an example from the last lesson:

```python
start = time.time()
ids = [expensive_task.remote(n) for n in range(5)]  # Fire off the asynchronous tasks
for n2, duration in ray.get(ids):                   # Retrieve all the values from the list of futures
    p(n2, duration)
pd(time.time() - start, prefix="Total time:")
```

Let's try it again with our new methods:

In [6]:
start = time.time()
array_ids = [make_array.remote(n*10) for n in range(5)]
added_array_ids = [add_arrays.remote(id, id) for id in array_ids]
for array in ray.get(added_array_ids):
    print(f'{array.size}: {array}')
pd(time.time() - start, prefix="Total time:")

0: []
10: [-0.00746376  1.51015548 -0.10346581  0.46925812 -3.18210131 -2.91379975
  3.80864868 -1.11616834  0.11588832 -3.73226179]
20: [ 1.54983569 -0.81686357  2.60341966 -0.42220452 -2.7620832   2.32624521
 -2.25088614  2.25651266 -2.54055303 -0.42301281 -0.67274209  0.04013319
  1.63945981  0.35830606 -1.62811846 -0.95519253  0.33825708  1.24650194
  1.32824537 -0.18716475]
30: [ 0.18799548  1.24683295 -2.18709975 -2.21858195  0.01928234 -1.31068506
 -1.01383147  4.06966548 -1.84651065  0.44773329 -0.028698    3.76742277
 -1.029437    0.54344869 -0.43253629 -1.20158117 -1.02505303  1.18738617
  0.14005394  2.69288496  3.50965665  0.09535116 -0.54410202 -0.99200292
  1.50944276  3.9124867  -1.72262906  0.74173623  3.96187394 -1.59007863]
40: [-0.7242384  -2.29397812  0.91929684  1.17995228 -2.06601651 -0.38821553
  1.17720197 -0.90278466 -3.7564516   3.37721051  0.29695218 -1.22411161
  4.1542128  -0.56829646 -1.04566615  2.29113107  1.80499828 -1.13590126
 -0.34820494 -2.77249547 

On my machine, I waited 8 seconds and then everything was printed at once.

There are two fundamental problems with the way we've used `ray.get()` so far:

1. There's no timeout, in case something gets "hung".
2. We have to wait for _all_ the objects to be available before `ray.get()` returns.

The ability to specify a timeout is essential in production code as a defensive measure. Many potential problems could happen in a real production system, any one of which could cause the task we're waiting on to take an abnormally long time to complete or never complete. Our application would be deadlocked waiting on this task. Hence, it's **strongly recommended** in production software to always use timeouts on blocking calls, so that the application can attempt some sort of recovery in situations like this, or at least report the error and "degrade gracefully".

Actually, there _is_ a `timeout=<value>` option you can pass to `ray.get()` ([documentation](https://ray.readthedocs.io/en/latest/package-ref.html#ray.get)), but it will most likely be removed in a future release of Ray. Why remove it if timeouts are important? This change will simplify the implementation of `ray.get()` and encourage the use of `ray.wait()` for waiting ([documentation](https://ray.readthedocs.io/en/latest/package-ref.html#ray.wait)) instead, followed by using `ray.get()` to retrieve values for tasks that `ray.wait()` tells us are finished. 

Using `ray.wait()` is also the way to fix the second problem with using `ray.get()` by itself, that we have to wait for all tasks to finish before we get any values back. Some of those tasks might finish quickly, like our contrived examples that sleep for short durations compared to other invocations. 

When you have a list of asynchronous tasks, you want to process the results of them as soon they become available, even while others continue to run. Use `ray.wait()` for this purpose.

Therefore, while `ray.get()` is simple and convenient, for _production code_, we recommend using `ray.wait()`, **with** timeouts, for blocking on running tasks. Then use `ray.get()` to retrieve values of completed tasks. Now we'll learn how to use these two together. For a longer discussion on `ray.wait()`, see [this blog post](https://medium.com/distributed-computing-with-ray/ray-tips-and-tricks-part-i-ray-wait-9ed7a0b9836d).

Here is the previous example rewritten to use `ray.wait()`:

In [7]:
start = time.time()
array_ids = [make_array.remote(n*10) for n in range(5)]
added_array_ids = [add_arrays.remote(id, id) for id in array_ids]

arrays = []
waiting_ids = list(added_array_ids)        # Assign a working list to the full list of ids
while len(waiting_ids) > 0:                # Loop until all tasks have completed
    # Call ray.wait with:
    #   1. the list of ids we're still waiting to complete,
    #   2. tell it to return immediately as soon as one of them completes,
    #   3. tell it wait up to 10 seconds before timing out.
    ready_ids, remaining_ids = ray.wait(waiting_ids, num_returns=1, timeout=10.0)
    print('Returned {:3d} completed tasks. (elapsed time: {:6.3f})'.format(len(ready_ids), time.time() - start))
    new_arrays = ray.get(ready_ids)
    arrays.extend(new_arrays)
    for array in new_arrays:
        print(f'{array.size}: {array}')
    waiting_ids = remaining_ids  # Reset this list; don't include the completed ids in the list again!
    
print(f"\nall arrays: {arrays}")
pd(time.time() - start, prefix="Total time:")

Returned   1 completed tasks. (elapsed time:  0.004)
0: []
Returned   1 completed tasks. (elapsed time:  2.010)
10: [ 1.83806876 -4.96113183  0.51568549 -3.72069353 -0.79526441  2.51089464
  0.07560527 -1.73313779 -0.27586426  1.36562054]
Returned   1 completed tasks. (elapsed time:  4.013)
20: [-1.11741918  0.30588653 -1.46338788 -0.9312212   2.67575836  2.05884653
  1.06104454 -0.16983274  3.12908962  0.88389943 -4.21013914  6.456811
  0.4912645   0.25362622  3.29218034  1.05894825 -0.50409912  4.10455236
 -2.67594573  0.0854673 ]
Returned   1 completed tasks. (elapsed time:  6.015)
30: [-3.57984093 -0.81773609 -4.20217648  0.13934324 -0.73219012  0.83268106
  0.60210076 -0.84958173 -0.43063911 -2.19425422  1.68319933  1.7211223
  0.73145622  3.16717009 -0.56633676  2.09332714 -2.95193651  1.14809513
  0.35415109  0.91777087  2.79533238  0.83691489  2.61802381 -3.34685928
  1.8350518  -1.74699111 -3.00123569 -1.88022392  0.25658696 -3.12006269]
Returned   1 completed tasks. (elapsed 

Now it still takes about 8 seconds to complete, 4 seconds for the longest invocation of `make_array` and 4 seconds for the invocation of `add_arrays`, but since the others complete more quickly, we see their results as soon as they become available, at 0, 2, 4, and 6 second intervals.

> **Warning:** For each call to `ray.wait()` in a loop like this, it's important to remove the ids that have completed. Otherwise, `ray.wait()` will return immediately with the same list containg the first completed item, over and over again; you'll loop forever!! Resetting the list is easy, since the second list returned by `ray.wait()` is the rest of the items that are still running. So, that's what we use.

Now let's try it with `num_returns = 2`:

In [8]:
start = time.time()
array_ids = [make_array.remote(n*10) for n in range(5)]
added_array_ids = [add_arrays.remote(id, id) for id in array_ids]

arrays = []
waiting_ids = list(added_array_ids)        # Assign a working list to the full list of ids
while len(waiting_ids) > 0:                # Loop until all tasks have completed
    # Call ray.wait with:
    #   1. the list of ids we're still waiting to complete,
    #   2. tell it to return immediately as soon as TWO of them complete,
    #   3. tell it wait up to 10 seconds before timing out.
    return_n = 2 if len(waiting_ids) > 1 else 1
    ready_ids, remaining_ids = ray.wait(waiting_ids, num_returns=return_n, timeout=10.0)
    print('Returned {:3d} completed tasks. (elapsed time: {:6.3f})'.format(len(ready_ids), time.time() - start))
    new_arrays = ray.get(ready_ids)
    arrays.extend(new_arrays)
    for array in new_arrays:
        print(f'{array.size}: {array}')
    waiting_ids = remaining_ids  # Reset this list; don't include the completed ids in the list again!
    
print(f"\nall arrays: {arrays}")
pd(time.time() - start, prefix="Total time:")

Returned   2 completed tasks. (elapsed time:  2.013)
0: []
10: [ 0.48505409 -2.08703879  1.33139831 -0.96397336  1.0099965   2.18098649
 -1.93973323  0.06238183  0.4540442   2.03157975]
Returned   2 completed tasks. (elapsed time:  6.010)
20: [ 3.46703945 -3.64272894 -0.39183821  1.24488608  0.15267756  1.00855068
  0.10061789  1.84261435 -1.38836964  2.37984148  4.67564279  0.4303022
 -2.02106666  1.01263646  2.12259866  1.44237974  0.04913943 -1.58249979
  0.42736538  1.60706635]
30: [-3.25126357 -1.21868205 -1.41183486 -0.78789907  0.5290288   3.6543227
  2.18011217 -1.83013921  1.76036858  5.62626519 -0.63223152 -1.8645452
 -2.30985655  3.78432571 -0.9774471   0.53505625 -0.20474847 -1.57810876
  0.65342757 -0.90194972 -0.52021854 -0.4161014   0.94925814  0.52472455
 -0.23822253 -0.92932931  3.07378238  0.10969498 -1.27984261  4.74758404]
Returned   1 completed tasks. (elapsed time:  8.009)
40: [ 2.7185277   0.87476927  0.31747447 -0.84252592  1.42260659  2.41055141
  2.12871571 -2

Now we get two at a time output. Note that we don't actually pass `num_returns=2` every time. If you ask for more items than the length of the input list, you get an error. So, we compute `num_returns`, using `2` except when there's only one task to wait on, in which case we use `1`. So, in fact, the output for `40` was a single task result, because we started with `5` and processed two at a time.

## Exercise 2

The following cell is identical to the last one. Modify it to use a timeout of `2.5` seconds, shorter than our longest tasks. What happens now? Try using other times.

In [9]:
start = time.time()
array_ids = [make_array.remote(n*10) for n in range(5)]
added_array_ids = [add_arrays.remote(id, id) for id in array_ids]

arrays = []
waiting_ids = list(added_array_ids)        # Assign a working list to the full list of ids
while len(waiting_ids) > 0:                # Loop until all tasks have completed
    # Call ray.wait with:
    #   1. the list of ids we're still waiting to complete,
    #   2. tell it to return immediately as soon as TWO of them complete,
    #   3. tell it wait up to 10 seconds before timing out.
    return_n = 2 if len(waiting_ids) > 1 else 1
    ready_ids, remaining_ids = ray.wait(waiting_ids, num_returns=return_n, timeout=10.0)
    print('Returned {:3d} completed tasks. (elapsed time: {:6.3f})'.format(len(ready_ids), time.time() - start))
    new_arrays = ray.get(ready_ids)
    arrays.extend(new_arrays)
    for array in new_arrays:
        print(f'{array.size}: {array}')
    waiting_ids = remaining_ids  # Reset this list; don't include the completed ids in the list again!
    
print(f"\nall arrays: {arrays}")
pd(time.time() - start, prefix="Total time:")

Returned   2 completed tasks. (elapsed time:  2.011)
0: []
10: [ 0.69182274  0.75867389  1.78036329 -0.87307422 -0.5442487  -0.16660188
 -5.19344479 -2.61279514 -1.35402879 -0.78964219]
Returned   2 completed tasks. (elapsed time:  6.015)
20: [-0.75825869 -0.04060735  2.62383677  2.00237471 -1.25487905 -0.84205285
 -1.44992225  2.57733202 -2.1633298   2.83485841  2.51319668  1.03793965
 -1.00110417 -1.57940368  1.32318568  2.5178875  -1.26190019  0.47609738
  2.5295883  -3.26841857]
30: [-0.39372451 -1.97998123 -1.89219034  0.94306718  2.61281957  1.74432369
  0.21304768 -1.9756494  -2.29869625  3.08188033  0.14693249 -4.43019766
  1.7351367   0.13475153 -3.11276612 -1.79539333 -2.6040562  -1.62332495
  1.28054004  2.00302429  2.31407782  0.08498056  0.07054166  0.85831582
  0.64051028  1.80203325 -0.35486994  1.102035   -0.2704127  -0.34755119]
Returned   1 completed tasks. (elapsed time:  8.015)
40: [ 0.63900911 -0.14046437  1.67720352  0.37711616 -3.6861706   3.14080746
  0.38948485

In conclusion:

> **Tips:**
>
> 1. Use `ray.wait()` with a timeout to wait for one or more running tasks. Then use `ray.get()` to retrieve the values for the finished tasks.
> 2. Don't ask for results you don't need.
> 3. Don't ask for the results you need until you really need them.

## Exercise 3

Let's make sure you understand how to use `ray.wait()`. The definitions from Exercise 1 in the previous lesson are repeated in the next cell. Change the definitions to use Ray. In particular, use `ray.wait()` as we used it above. You can just use the default values for `num_returns` and `timeout` if you want. The second cell uses `assert` statements to check your work.

> **Tip:** The solution is in the `solutions` folder.

In [10]:
def slow_square(n):
    time.sleep(n)
    return n*n

start = time.time()
squares = [slow_square(n) for n in range(4)]
duration = time.time() - start

In [11]:
assert squares == [0, 1, 4, 9], f'Did you use ray.get() to retrieve the values? squares = {squares}'
assert duration < 4.1, f'Did you use Ray to parallelize the work? duration = {duration}' 

AssertionError: Did you use Ray to parallelize the work? duration = 6.007925987243652

## What Is the Optimal Task Granularity

How fine-grained should Ray tasks be? There's no fixed rule of thumb, but Ray clearly adds some overhead for task management and using object stores in a cluster. Therefore, it makes sense that tasks which are too small will perform poorly.

We'll explore this topic over several more lessons, but for now, let's get a sense of the overhead while running in your setup.

We'll continue to use NumPy arrays to create "load", but remove the `sleep` calls:

In [12]:
def noop(n):
    return n

def local_make_array(n):
    return np.random.standard_normal(n)

@ray.remote
def remote_make_array(n):
    return local_make_array(n)

Let's do `trials` runs for each experiment, to average out background noise:

In [13]:
trials=100

First, let's use `noop` to baseline local function calls. Note that we call `print` for the duration, rathern than `pd`, because the overhead is so low the `pd` formatting will print `0.000`:

In [14]:
start = time.time()
[noop(t) for t in range(trials)]
print(f'{time.time() - start} seconds')

0.00013589859008789062 seconds


Let's try the same run with `local_make_array(n)` for `n = 100000`:

In [15]:
start = time.time()
[local_make_array(100000) for _ in range(trials)]
print(f'{time.time() - start} seconds')

0.27163124084472656 seconds


So, we can safely ignore the "noop" overhead for now. For completeness, here's what happens with remote execution:

In [16]:
start = time.time()
ids = [remote_make_array.remote(100000) for _ in range(trials)]
ray.get(ids)
print(f'{time.time() - start} seconds')

0.10001182556152344 seconds


For arrays of 100000, using Ray is faster (at least on this test machine). The benefits of parallel computation, rather than synchronous, already outweight the Ray overhead.

So, let's run some trials with increasingly large array sizes, to compare the performance with local vs. remote execution. First, we'll set up `matplotlib`:

In [17]:
local_durations = []
remote_durations = []
# These n values were determined by experimentation on this test machine. 
# If you are using an old machine, and this cell takes a long time to execute,
# you could set the `trials` value above to a smaller number. 
ns = [i*(10**j) for j in range(2,5) for i in [1,2,3,5,8]]
for n in ns:
    start_local = time.time()
    [local_make_array(n) for _ in range(trials)]
    local_durations.append(time.time() - start_local)
    
    start_remote = time.time()
    ids = [remote_make_array.remote(n) for _ in range(trials)]
    ray.get(ids)
    remote_durations.append(time.time() - start_remote)
(ns, local_durations, remote_durations)

([100,
  200,
  300,
  500,
  800,
  1000,
  2000,
  3000,
  5000,
  8000,
  10000,
  20000,
  30000,
  50000,
  80000],
 [0.001489877700805664,
  0.0010769367218017578,
  0.0009431838989257812,
  0.0014388561248779297,
  0.002299070358276367,
  0.0028460025787353516,
  0.0050029754638671875,
  0.008249998092651367,
  0.015505075454711914,
  0.024491071701049805,
  0.03491997718811035,
  0.05433201789855957,
  0.07851409912109375,
  0.13427305221557617,
  0.2106318473815918],
 [0.028757095336914062,
  0.022654056549072266,
  0.0196988582611084,
  0.01858806610107422,
  0.01920795440673828,
  0.019514799118041992,
  0.01967000961303711,
  0.02035689353942871,
  0.026332378387451172,
  0.02686476707458496,
  0.027003765106201172,
  0.03647923469543457,
  0.0460667610168457,
  0.05334877967834473,
  0.08004593849182129])

In [27]:
import numpy as np

from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_file, show

import bokeh.io
# The next two lines prevent Bokeh from opening the graph in a new window.
bokeh.io.reset_output()
bokeh.io.output_notebook()

tooltips = [
    ("name", "$name"),
    ("array size", "$x"),
    ("time", "$y")]
p1 = figure(x_axis_type="log", y_axis_type="log", title="Execution Times", tooltips=tooltips)
p1.grid.grid_line_alpha=0.3
p1.xaxis.axis_label = 'array size'
p1.yaxis.axis_label = 'time'

p1.line(ns, local_durations, color='#A6CEE3', legend_label='local', name='local')
p1.circle(ns, local_durations, color='darkgrey', size=4)
p1.line(ns, remote_durations, color='#B2DF8A', legend_label='remote', name='remote')
p1.square(ns, remote_durations, color='darkgrey', size=4)
p1.legend.location = "top_left"

show(gridplot([[p1]], plot_width=800, plot_height=400))

Loading BokehJS ...

Let's confirm what the graph shows as the crossing point:

In [19]:
i=0
while i < len(ns) and local_durations[i] < remote_durations[i]:
    i=i+1
print('The Ray times are faster starting at n = {:d}, local = {:6.3f} vs. remote = {:6.3f}'.format(
    ns[i], local_durations[i], remote_durations[i]))

The Ray times are faster starting at n = 10000, local =  0.035 vs. remote =  0.027


**TODO:** add an exploration of tasks/second limits?

## How Distributed Task Management Works

> **Note:** If you just want to learn the Ray API, you can safely skip the rest of this lesson (notebook) for now. It continues the exploration of how Ray works internally, which we started in the previous lesson. However, you should come back to this material at some point, so you'll develop a better understanding of how Ray works.

At the end of the last lesson, we examined Ray task scheduling at a high-level, by watching the Ray Dashboard and analyzing the performance times. Now we'll walk through some images that show the process Ray follows to place tasks around a cluster. 

Assume we will invoke the `make_array` task twice, then invoke `add_arrays` to sum the returned NumPy arrays. Graphically, it looks as follows:
![Ray under the hood 1](../images/Ray-Cluster/Ray-Cluster.001.jpeg)

How does this get scheduled in a cluster? Here we'll assume a three-node cluster that has resources for running two Ray worker tasks per node (under powered compared to what we learned using Ray Dashboard last lesson!).
![Ray under the hood 2](../images/Ray-Cluster/Ray-Cluster.002.jpeg)

First, assume that the driver program is running on Node1. So it will invoke the local scheduler to schedule the three tasks.
![Ray under the hood 3](../images/Ray-Cluster/Ray-Cluster.003.jpeg)

Immediately the ids for the task futures are returned. The _Global Control Store_ tracks where every task is running and every object is stored in the local _Object Stores_.
![Ray under the hood 4](../images/Ray-Cluster/Ray-Cluster.004.jpeg)

Suppose the local scheduler has available capacity in the first worker on the same node. It schedules the first `make_array` task there.
![Ray under the hood 5](../images/Ray-Cluster/Ray-Cluster.005.jpeg)

It decides to schedule the second `make_array` task in a worker on node 2.
![Ray under the hood 6](../images/Ray-Cluster/Ray-Cluster.006.jpeg)

When the two tasks finish, they place their result objects in their local object stores.
![Ray under the hood 7](../images/Ray-Cluster/Ray-Cluster.007.jpeg)

Now `add_array` can be scheduled, because the two tasks it depends on are done. Let's suppose it gets scheduled in the second worker on Node 1.
![Ray under the hood 8](../images/Ray-Cluster/Ray-Cluster.008.jpeg)

The first object it needs is already on the same node, in the object store, so the `add_arrays` task can _read it directly from shared memory_. No copying is required to the worker's process space.
![Ray under the hood 9](../images/Ray-Cluster/Ray-Cluster.009.jpeg)

However, the second object is on a different node, so Ray copies it to the local object store. 
![Ray under the hood 10](../images/Ray-Cluster/Ray-Cluster.010.jpeg)

Now it can also be read from shared memory.
![Ray under the hood 11](../images/Ray-Cluster/Ray-Cluster.011.jpeg)

When `add_arrays` is finished, it writes its results to the local object store.
![Ray under the hood 12](../images/Ray-Cluster/Ray-Cluster.012.jpeg)

At this point, if the driver calls `ray.get(id3)`, it will return `obj3`.
![Ray under the hood 13](../images/Ray-Cluster/Ray-Cluster.013.jpeg)

Whew! Hopefully you have a better sense of what Ray does under the hood. Scheduling tasks on other nodes and copying objects between object stores is efficient, but incurs unavoidable network overhead.